In [176]:
#dependencies
import pandas as pd 
import gmaps
import requests
import matplotlib.pyplot as plt 
from config import g_key

In [177]:
#read in csv file and convert to dataFrame
weatherDF = pd.read_csv("weather_data/weather_database.csv")
#configure gmaps to use my google api key
gmaps.configure(api_key=g_key)

In [178]:
#retrieve customer preferences
custMinTemp = float(input("What is the minimum temperature you would like for your trip? "))
custMaxTemp = float(input("What is the maximum temperature you would like for your trip? "))
custRainPref = input("Do you want it to be raining? (yes/no) ")
custSnowPref = input("Do you want it to be snowing? (yes/no) ")

In [179]:
#filter dataframe based on customer inputs
prefCityDF = weatherDF.loc[(weatherDF["Max Temp"] <= custMaxTemp) & (weatherDF["Max Temp"] >=custMinTemp)]

if custRainPref =="yes":
    prefCityDF = prefCityDF[prefCityDF["Rain (inches)"] !=0]
else:
    prefCityDF = prefCityDF[prefCityDF["Rain (inches)"] ==0]
if custSnowPref == "yes":
    prefCityDF = prefCityDF[prefCityDF["Snow (inches)"] !=0]
else:
    prefCityDF = prefCityDF[prefCityDF["Snow (inches)"] ==0]

In [180]:
#create hotel name column
prefCityDF["Hotel Name"] = ""

In [181]:
#hotel search parameters
params = {"radius": 5000, "type":"lodging", "key":g_key}
#iterate thru the dataframe
for index, row in prefCityDF.iterrows():
    #get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    #add lat and lng to location key for params dict
    params["location"] = f"{lat},{lng}"
    #use search term lodging and lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make request and get the json data from search
    hotels = requests.get(base_url, params=params).json()
    #grab first hotel from the results and store the name
    try:
        prefCityDF.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [182]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [183]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in prefCityDF.iterrows()]

In [184]:
# add a heatmap of temp for vacation spots
locations = prefCityDF[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [185]:
#create output file (csv)
output_data_file = "weather_data/weatherPy_vacation.csv"
#export city_data into a csv
prefCityDF.to_csv(output_data_file, index_label="City_ID")